# Chapter 4: MNIST Basics

## Setup

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

## Foundations of Computer Vision

### Fun facts from history

- Lenet-5 was the first computer system to recognise handwritten digits well enough for practical use. 
- Yann LeCun, Yoshua Bengio and Geoffrey Hinton were awarded the Turing Award for persevering with NNs during the second AI winter. Even papers with SoTA results were rejected for using NNs!
- Jurgen Schmidhuber and Sepp Hochreiter pioneered LSTMs (among other things)
- In 1974, Paul Werbos invented backprop

### Simple classification - 2 categories

We start with a simple task - classifying a digit as a 3 or a 7.

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
(path/"train").ls()

In [ ]:
threes = (path/"train"/"3").ls().sorted()
sevens = (path/"train"/"7").ls().sorted()
threes

In [ ]:
# see sample image file
im3_path = threes[1] 
im3 = Image.open(im3_path) # uses PIL
im3

In [ ]:
# view im3 as array
array(im3)[4:10, 4:10]

In [ ]:
# view part of im3 as tensor
tensor(im3)[4:10,4:10]

In [ ]:
# slice array to get top part of digit
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[4:15,4:22])
df.style.set_properties(**{"font-size":"6pt"}).background_gradient("Greys")

How does the model learn to recognise the digits? Personal guesses: 
- It does some kind of edge detection by looking at boundaries of bright vs dark pixels
- It puts these together by trying to maintain a continuous curve and seeing how it fits

As a first attempt, we might try to use pixel similarity. Take the average value of the pixels of the 3s, and do the same for the 7s. Then classify by seeing which of the two the image is most similar to.

This can act as a baseline, i.e. a model which you are confident should perform reasonably well. It should be easy to implement and test, and used to compare with further attempts. 

In [ ]:
# stack 3s and 7s and do sanity check
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors), len(seven_tensors)

In [ ]:
show_image(three_tensors[1]); # semicolon suppresses displaying the type

Now we stack the image tensors and take the means

In [ ]:
# stack the tensors
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_threes.shape

In [ ]:
# length of a tensor shape is its rank
len(stacked_threes.shape)

In [ ]:
# can also use ndim to get the rank directly
stacked_threes.ndim

In [ ]:
mean3 = stacked_threes.mean(0)
show_image(mean3);

This is the "ideal" number 3 - it is clearer where the images agree it should be, but blurry where the images disagree. 

In [ ]:
mean7 = stacked_sevens.mean(0)
show_image(mean7);

If we now pick a random 3 and determine the distance between this and the ideal 3 (e.g. using absolute values of squared distances), then we get: 

In [ ]:
# choose a random 3
a_3 = stacked_threes[1]
show_image(a_3);

In [ ]:
dist_3_abs = (a_3 - mean3).abs().mean()
dist_3_sqr = ((a_3 - mean3)**2).mean().sqrt()
dist_3_abs,dist_3_sqr 

In [ ]:
dist_7_abs = (a_3 - mean7).abs().mean()
dist_7_sqr = ((a_3 - mean7)**2).mean().sqrt()
dist_7_abs, dist_7_sqr

The distance is smaller for the 3, so in this case the model gives the right prediction. Let's do the same thing using loss functions:

In [ ]:
F.l1_loss(a_3.float(),mean7), F.mse_loss(a_3,mean7).sqrt()

MSE penalises larger losses more and is more lenient with smaller ones, compared to L1 loss.

### NumPy vs PyTorch

NumPy is very similar to PyTorch except that it doesn't support GPU usage or gradient calulations. 

Numpy arrays can have data of any type, including arrays (with potentially different sizes, forming a "jagged array"). This isn't possible in PyTorch - the structure must be rectangular, and all components must have the same numeric type. 

Python is generally pretty slow - things that are fast (e.g. in Numpy or Pytorch) are usually written in *other languages*, and can often do calculations thousands of times faster than pure Python. Numpy and PyTorch make use of this to make it much faster, so avoid writing loops in Python and instead use matrix operations that are optimised in lower-level languages. 

### Computing metrics using broadcasting

The previous model has no learned components, so overfitting isn't really a problem. But just for practice, let's calculate some metrics using a validation set.  

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/"valid"/"7").ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape, valid_7_tens.shape

Note to self: get in the habit of checking shapes as you go!

In [ ]:
# function to find mean abs error
def mnist_distance(a,b): return (a-b).abs().mean((-1,-2))
mnist_distance(a_3, mean3)

What happens if we feed in two tensors of different dimensions?

In [ ]:
valid_3_dist = mnist_distance(valid_3_tens,mean3)
valid_3_dist, valid_3_dist.shape

We get a rank-1 tensor of length 1010. How did this happen?

In the subtraction operation `a-b`, PyTorch did *broadcasting* to make the smaller rank tensor (in this case `mean3`) have the same size as the larger rank one. This doesn't actually involve copying the smaller tensor multiple times (this would take up tons of memory); it just pretends to give a tensor of the desried shape. The whole calculation is done in C, or CUDA for GPUs. 

Let's now define a function for checking is an image is closer to the ideal 3 or ideal 7.

In [ ]:
def is_3(x): return mnist_distance(x,mean3) < mnist_distance(x,mean7)

In [ ]:
is_3(a_3), is_3(a_3).float()

In [ ]:
is_3(valid_3_tens)

Now let's calculate the accuracy.

In [ ]:
accuracy_3s = is_3(valid_3_tens).float().mean()
accuracy_7s = (1 - is_3(valid_7_tens).float()).mean()

accuracy_3s, accuracy_7s, (accuracy_3s + accuracy_7s)/2

## Stochastic Gradient Descent

Instead of looking at similarities between an image and an ideal image, we can try to figure out parameters relating to each pixel. The highest weights are associated with a particular category.

Training a model follows the rough process below:
- Initialise: typically set the parameters to random values
- Loss: small if the model does well, large if it does poorly
- Step: to change the parameters based on the loss and using calculus
- Stop: decide how many epochs to the train the model (e.g. by iterating the training process until you don't want to wait any longer, or if the model is already good enough)

Potentially confusing point: in deep learning, "gradients" typically mean the *value* of a function's derivative for a given argument, rather than the function itself. 

PyTorch calculates derivatives really quickly, and automatically. 

In [ ]:
def f(x): return x**2

In [ ]:
xt = tensor(3.).requires_grad_() # tells PyTorch to calculate gradients

In [ ]:
yt = f(xt)
yt

In [ ]:
yt.backward() # backprop

In [ ]:
# view gradients
xt.grad

In [ ]:
# repeat previous steps with a vector argument
xt = tensor([3., 4., 10.]).requires_grad_()
xt

In [ ]:
def f(x): return (x**2).sum()

yt = f(xt)
yt

In [ ]:
yt.backward()
xt.grad

In [ ]:
xt

### End-to-end example

Imagine measuring the speed of a roller coaster as it goes over the top of a hump. How doees it change over time? Let's measure speed manually every second for 20 seconds.

In [ ]:
time = torch.arange(0,20).float(); time

In [ ]:
speed = torch.randn(20)*3 + 0.75*(time-9.5)**2 + 1
plt.scatter(time, speed);

Now we use SGD and assume a speed function that is quadratic in time.

In [ ]:
# clearly separate the function's input and its parameters
def f(t, params):
  a,b,c = params
  return a*(t**2) + (b*t) + c

In [ ]:
# MSE is common for continuous data
def mse(preds, targets): return ((preds - targets)**2).mean().sqrt()

Step 1: initialise the parameters

In [ ]:
params = torch.randn(3).requires_grad_()

In [ ]:
orig_params = params.clone()

Step 2: calculate predictions

In [ ]:
preds = f(time, params)

In [ ]:
def show_preds(preds, ax=None):
  if ax is None: ax=plt.subplots()[1]
  ax.scatter(time, speed)
  ax.scatter(time, to_np(preds), color="red")
  ax.set_ylim(-300,100)

In [ ]:
show_preds(preds)

Step 3: calculate the loss

In [ ]:
loss = mse(preds, speed)
loss

Step 4: calculate the gradients

In [ ]:
loss.backward()
params.grad

In [ ]:
params.grad * 1e-3

In [ ]:
params

Step 5: step the weights

In [ ]:
lr = 1e-3
params.data -= lr * params.grad.data
params.grad = None

# this requires function composition, 
# which allows us to do backprop via the chain rule

In [ ]:
preds = f(time, params)
mse(preds, speed)

In [ ]:
show_preds(preds)

In [ ]:
# define function to repeatedly apply the above process
def apply_step(params, prn=True):
  preds = f(time, params)
  loss = mse(preds, speed)
  loss.backward()
  params.data -= lr * params.grad.data 
  params.grad = None
  if prn: print(loss.item())
  return preds

Step 6: repeat

In [ ]:
for i in range(10): apply_step(params)

In [ ]:
params = orig_params.detach().requires_grad_()

In [ ]:
# show visually
_,axs = plt.subplots(1,4,figsize=(12,3))
for ax in axs: show_preds(apply_step(params, False),ax)
plt.tight_layout()

Step 7: stop  
In practice we would do this using training and validation losses. 

## The MNIST Loss Function

We want to apply SGD to MNIST, but we'll need a different loss function. What should we use?

In [ ]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

In [ ]:
# define labels for images - 1 for 3s and 0 for 7s
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
train_x.shape, train_y.shape

Let's turn this into a dataset of IDV, DV pairs.

In [ ]:
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape, y

In [ ]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))

In [ ]:
# randomise weights
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [ ]:
weights = init_params((28*28,1))

In [ ]:
bias = init_params(1)

In [ ]:
# make prediction for one image y = wx+b
(train_x[0]*weights.T).sum() + bias

In [ ]:
# matrix multiplication for weights * pixels
def linear1(xb): return xb@weights + bias
preds = linear1(train_x)
preds

In [ ]:
# check accuracy - see if output is 3 or 7
corrects = (preds>0.5).float() == train_y
corrects

In [ ]:
corrects.float().mean().item()

In [ ]:
trgts = tensor([1,0,1])
prds = tensor([0.9,0.4,0.2])

In [ ]:
def mnist_loss(predictions, targets):
  return torch.where(targets==1, 1-predictions, predictions).mean()

In [ ]:
torch.where(trgts==1, 1-prds, prds)

This measures how close the prediction is to 1 if it should be 1, and similarly for 0.

In [ ]:
mnist_loss(prds,trgts)

What if the outputs aren't between 0 and 1? For this we can *ensure* that this happens using a sigmoid function. 

In [ ]:
plot_function(torch.sigmoid, title="Sigmoid", min=-4, max=4);

Let's update our function based on this.

In [ ]:
def mnist_loss(predictions, targets):
  predictions = predictions.sigmoid()
  return torch.where(targets==1, 1-predictions, predictions).mean()

Notice that this loss function is smooth, so it has a meaningful derivative and can be used to update the model's parameters. 

To do minibatch gradient descent, we want to shuffle the dataset. This is done in the `DataLoader`.

In [ ]:
col1 = range(15)
dl = DataLoader(col1, batch_size=5, shuffle=True)
list(dl)

In [ ]:
ds = L(enumerate(string.ascii_lowercase))
ds

In [ ]:
dl = DataLoader(ds, batch_size=6, shuffle=True)
list(dl)

## Putting it all together

In [ ]:
# reinitialise params
weights = init_params((28*28,1))
bias = init_params(1)

In [ ]:
dl = DataLoader(dset, batch_size=256)
xb,yb = first(dl)
xb.shape, yb.shape

In [ ]:
# dataloader for validation set
valid_dl = DataLoader(valid_dset, batch_size=256)

In [ ]:
batch = train_x[:4]
batch.shape

In [ ]:
preds = linear1(batch)
preds

In [ ]:
loss = mnist_loss(preds, train_y[:4])
loss

In [ ]:
loss.backward()
weights.grad.shape,weights.grad.mean(),bias.grad

In [ ]:
def calc_grad(xb,yb,model):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  loss.backward()

In [ ]:
calc_grad(batch,train_y[:4],linear1)
weights.grad.mean(),bias.grad

In [ ]:
calc_grad(batch,train_y[:4],linear1)
weights.grad.mean(),bias.grad

In [ ]:
# we need to set the gradients to zero first,
# else loss.backward adds the gradients to stored gradients
weights.grad.zero_()
bias.grad.zero_();

In [ ]:
def train_epoch(model, lr, params):
  for xb,yb in dl:
    calc_grad(xb,yb,model)
    for p in params:
      p.data -= p.grad*lr
      p.grad.zero_()

In [ ]:
# check accuracy on validation set
def batch_accuracy(xb, yb):
  preds = xb.sigmoid()
  correct = (preds>0.5) == yb
  return correct.float().mean()

In [ ]:
batch_accuracy(linear1(batch), train_y[:4])

In [ ]:
def validate_epoch(model):
  accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
  return round(torch.stack(accs).mean().item(),4)

In [ ]:
validate_epoch(linear1)

In [ ]:
# start training for an epoch and
# see if accuracy changes
lr = 1.
params = weights, bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

In [ ]:
for i in range(20):
  train_epoch(linear1,lr,params)
  print(validate_epoch(linear1), end=" ")

## Creating an optimiser

In [ ]:
linear_model = nn.Linear(28*28, 1)

In [ ]:
w,b = linear_model.parameters()
w.shape, b.shape

In [ ]:
# use the above info to make the optimiser
class BasicOptim: 
  def __init__(self, params, lr): self.params, self.lr = list(params), lr

  def step(self, *args, **kwargs):
    for p in self.params: p.data -= p.grad.data * self.lr 
  
  def zero_grad(self, *args, **kwargs):
    for p in self.params: p.grad = None

In [ ]:
opt = BasicOptim(linear_model.parameters(), lr)

In [ ]:
def train_epoch(model):
  for xb, yb in dl:
    calc_grad(xb, yb, model)
    opt.step()
    opt.zero_grad()

In [ ]:
validate_epoch(linear_model)

In [ ]:
def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end = " ")

In [ ]:
train_model(linear_model, 20)

In [ ]:
# try using fastai's SGD class
linear_model = nn.Linear(28*28, 1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)

Alternative approach: use `Learner.fit` instead of `train_model`.

In [ ]:
dls = DataLoaders(dl, valid_dl)

In [ ]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(10, lr=lr)

## Adding nonlinearity

Instead of a linear classifier, let's use a neural net

In [ ]:
def simple_net(xb):
  res = xb@w1 + b1
  res = res.max(tensor(0.0))
  res = res@w2 + b2
  return res

In [ ]:
w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,1))
b2 = init_params(1)

In [ ]:
# introduce nonlinearity using ReLU
plot_function(F.relu)

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [ ]:
learn = Learner(dls, simple_net, opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(40, 0.1)

In [ ]:
plt.plot(L(learn.recorder.values).itemgot(2));

In [ ]:
# final accuracy
learn.recorder.values[-1][2]

We can add as many linear layers as we want, as long as we add nonlinearities between them.

## Questionnaire

1. **How is a grayscale image represented on a computer? How about a color image?**  
Grayscale images are stored in arrays, where each pixel has a single value. For colour images, this is typically three values instead, representing red, green and blue.

1. **How are the files and folders in the `MNIST_SAMPLE` dataset structured? Why?**  
Separate folders for the training and validation sets, which are further separated into categories (based on the number in the image).

1. **Explain how the "pixel similarity" approach to classifying digits works.**  
It takes all the images of a particular number and takes the mean of their pixel values to get an "ideal image". To classify an image, subtract the image's pixel values from the ideal image's pixel values - the category with the smallest difference is chosen as the prediction.

1. **What is a "rank-3 tensor"?**  
A tensor with "three layers of nested lists"

1. **What is the difference between tensor rank and shape? How do you get the rank from the shape?**  
Tensor rank is the length of the shape (the number of axes in the tensor), while the shape tells you the length along each axis.

1. **What are RMSE and L1 norm?**  
RMSE is the Euclidean norm. The L1 norm is the absolute value. 

1. **How can you apply a calculation on thousands of numbers at once, many thousands of times faster than a Python loop?**  
Use vectorisation. In libraries like NumPy and PyTorch, this is implemented in C (or other low-level languages, e.g. CUDA for GPUs) so it's much more efficient.

1. **What is broadcasting?**  
Expanding a tensor with smaller rank such that it has the same size as a larger one, e.g. for an arithmetic operation.

1. **Are metrics generally calculated using the training set, or the validation set? Why?**  
The validation set. We want to see how well it performs on data that wasn't directly used for training, so that we can test generalisation.

1. **What is SGD?**  
A way of updating the parameters of a model. It does this by calculating the gradient of the loss using a stochastically chosen data point, then updating the weights using the calculated derivative.

1. **Why does SGD use mini-batches?**  
It's less noisy than updating with a single data point, and faster than going through the whole dataset. It's also more parallelisable than using a single data point. 

1. **What are the seven steps in SGD for machine learning?**  
Initialise, predict, find loss, find gradient, step, repeat from prediction, iterate until it's time to stop.

1. **How do we initialize the weights in a model?**  
Often randomly.

1. **What is "loss"?**  
A value that is used to update the parameters of a model; it shows the difference between predictions and actual target values.

1. **Why can't we always use a high learning rate?**  
We may not converge to the global optimum (or any optimum).

1. **What is a "gradient"?**  
A value (in deep learning) that is essentially just the derivative of the loss. 

1. **Do you need to know how to calculate gradients yourself?**  
Not really, just use the in-built tools. But you should know it!

1. **Why can't we use accuracy as a loss function?**  
It doesn't change with small changes in parameters, so it's hard for the model to learn. On the other hand, something like MSE loss changes slightly with every step.

1. **Draw the sigmoid function. What is special about its shape?**  
S shape. It squishes the outputs between 0 and 1.

1. **What is the difference between a loss function and a metric?**  
A loss function is used for updating parameters. A metric is used for checking performance and not for training.

1. **What is the function to calculate new weights using a learning rate?**  
The step function in the optimiser

1. **What does the `DataLoader` class do?**  
It takes a Python collection and turns it into an iterator. It allows us to read images from a dataset. 

1. **What does `view` do in PyTorch?**  
It changes the shape of a tensor without changing its contents.

1. **What are the "bias" parameters in a neural network? Why do we need them?**  
Terms that make outputs (before nonlinear activation) affine transformed rather than just linearly transformed. They help us shift the activation - else the output would always be zero if the input is zero.

1. **What does the `@` operator do in Python?**  
Matrix multiplication.

1. **What does the `backward` method do?**  
It calculates the gradients using backprop.

1. **Why do we have to zero the gradients?**  
Else newly calculated gradients would get added to previous ones.

1. **What information do we have to pass to `Learner`?**  
Dataloaders, model, optimisation function, loss function, desired metrics

1. **What is an "activation function"?**  
A nonlinear function that takes the activation as input, allowing the model to learn nonlinear relationships. 

1. **What's the difference between `F.relu` and `nn.ReLU`?**  
They do the same thing, but `nn.ReLU` is a PyTorch module whereas `F.relu` is a function.

1. **The universal approximation theorem shows that any function can be approximated as closely as needed using just one nonlinearity. So why do we normally use more?**  
It's less computationally demanding, and it's also easier to learn more complex functions this way. Don't make the mistake of the second AI winter!